In [1]:
import csv

In [2]:
lng_des = []
ndb_nos = []
flag = 1
with open('./FOOD_DES.csv','rt')as f:
  data = csv.reader(f)
  for row in data:
        if flag == 0:
            if ("Babyfood") in row[2]:
                continue
            ndb_nos.append(row[0])
            lng_des.append(row[2])
        else:
            flag = 0

In [3]:
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

In [5]:
import nltk
import re
sing_names = []
lemm_lng_des = []

for i,d in enumerate(lng_des):
    word_list = nltk.word_tokenize(d)
    lemmatized_output = ' '.join([wnl.lemmatize(w.lower()) for w in word_list])
    lemm_lng_des.append(re.sub(r'([^.a-zA-Z0-9\s\,])', '', lemmatized_output))

lemm_lng_des = list(filter(lambda x: ("babyfood") not in x.lower(), lemm_lng_des))

names_only = list(map(lambda x: {
    '0' : x.split(',')[0].lower().strip(),
    '1' : x.split(',')[1].lower().strip() if len(x.split(',')) > 1 else "",
    '2' : x.split(',')[2].lower().strip() if len(x.split(',')) > 2 else "",
}, lemm_lng_des))

sing_names = list(map(lambda x: wnl.lemmatize(x.split(',')[0].lower()) , lng_des))
states_only = list(map(lambda x: ','.join(x.lower().split(',')[1:]) , lng_des))


In [6]:
for i,d in enumerate(names_only):
#     print(names_only[i])
    names_only[i]['0-1'] = (names_only[i]['0'] + " " + names_only[i]['1']).strip()
    names_only[i]['0-2'] = (names_only[i]['0'] + " " + names_only[i]['2']).strip()
    names_only[i]['1-2'] = (names_only[i]['1'] + " " + names_only[i]['2']).strip()
    names_only[i]['0-2'] = (names_only[i]['0'] + " " + names_only[i]['1'] + " " + names_only[i]['2']).strip()

In [7]:
from nltk.stem import WordNetLemmatizer
import nltk
import textdistance
wnl = WordNetLemmatizer()

In [146]:
def findIng(ing_name, ing_state=None):
    word_list = nltk.word_tokenize(re.sub(r'([^.a-zA-Z\s\,])', '', ing_name.lower()))
    ing_name = ' '.join([wnl.lemmatize(w) for w in word_list]).strip()
    for i,d in enumerate(names_only):
        if textdistance.jaccard(ing_name.split(), d['0'].split()) == 1.0:
            return i
    for i,d in enumerate(names_only):
        if textdistance.jaccard(ing_name.split(), d['1'].split()) == 1.0:
            return i
        if textdistance.jaccard(ing_name.split(), d['0-1'].split()) == 1.0:
            return i
    for i,d in enumerate(names_only):
        if textdistance.jaccard(ing_name.split(), d['2'].split()) == 1.0:
            return i
        if textdistance.jaccard(ing_name.split(), d['0-2'].split()) == 1.0:
            return i
        if textdistance.jaccard(ing_name.split(), d['1-2'].split()) == 1.0:
            return i
    
    maxSoFar = 0
    maxi = -1
    for i,d in enumerate(names_only):
        if textdistance.jaccard(ing_name.split(), d['0'].split()) > maxSoFar:
            maxSoFar = textdistance.jaccard(ing_name.split(), d['0'].split())
            maxi = i
        if textdistance.jaccard(ing_name.split(), d['1'].split()) > maxSoFar:
            maxSoFar = textdistance.jaccard(ing_name.split(), d['1'].split())
            maxi = i
        if textdistance.jaccard(ing_name.split(), d['0-1'].split()) > maxSoFar:
            maxSoFar = textdistance.jaccard(ing_name.split(), d['0-1'].split())
            maxi = i
        if textdistance.jaccard(ing_name.split(), d['2'].split()) > maxSoFar:
            maxSoFar = textdistance.jaccard(ing_name.split(), d['2'].split())
            maxi = i
        if textdistance.jaccard(ing_name.split(), d['0-2'].split()) > maxSoFar:
            maxSoFar = textdistance.jaccard(ing_name.split(), d['0-2'].split())
            maxi = i
        if textdistance.jaccard(ing_name.split(), d['1-2'].split()) > maxSoFar:
            maxSoFar = textdistance.jaccard(ing_name.split(), d['1-2'].split())
            maxi = i
    return maxi

In [14]:
def DistBatra(str1, str2):
    str1 = set(str1)
    str2 = set(str2)
#     print (str1, str2)
#     print (str1 & str2)
    return float(len(str1 & str2)) / len(str1)

def findIng2(ing_name, ing_state=None):
    word_list = nltk.word_tokenize(re.sub(r'([^.a-zA-Z\s\,])', '', ing_name.lower()))
    ing_name = ' '.join([wnl.lemmatize(w) for w in word_list]).strip()

    foundD0 = []
    foundD1 = []
    foundD01 = []
    foundD2 = []
    foundD02 = []
    foundD12 = []
    priorityOrder = []
    d0P = False
    d1P = False
    d01P = False
    d02P = False
    d12P = False
    d2P = False
    for i,d in enumerate(names_only):
        if DistBatra(ing_name.split(), d['0'].split()) == 1.0:
            foundD0.append(i)
            if d0P == False:
                d0P = True
        if DistBatra(ing_name.split(), d['1'].split()) == 1.0:
            foundD1.append(i)
            if d1P == False:
                d1P = True
        if DistBatra(ing_name.split(), d['0-1'].split()) == 1.0:
            foundD01.append(i)
            if d01P == False:
                d01P = True
        if DistBatra(ing_name.split(), d['2'].split()) == 1.0:
            foundD2.append(i)
            if d2P == False:
                d2P = True
        if DistBatra(ing_name.split(), d['0-2'].split()) == 1.0:
            foundD02.append(i)
            if d02P == False:
                d02P = True
        if DistBatra(ing_name.split(), d['1-2'].split()) == 1.0:
            foundD12.append(i)
            if d12P == False:
                d12P = True
    if d0P:
        priorityOrder.append(foundD0)
    if d1P:
        priorityOrder.append(foundD1)
    if d01P:
        priorityOrder.append(foundD01)
    if d2P:
        priorityOrder.append(foundD2)
    if d12P:
        priorityOrder.append(foundD12)
    if d02P:
        priorityOrder.append(foundD02)
    
    maxSoFar = 0
#     for arr in priorityOrder:
#         for item in arr:
#             print(lng_des[item])
    if ing_state == None:
        ing_state = "raw"

    if ing_state:
        filterWithState = list(map(lambda arr: list(filter(lambda ind: DistBatra(ing_state.lower().split(), lemm_lng_des[ind].split()) == 1,arr)),priorityOrder))
        for arr in filterWithState:
            if (len(arr) == 0):
                continue
            for item in arr:
                return item
        
        ing_state = "raw"
        filterWithState = list(map(lambda arr: list(filter(lambda ind: DistBatra(ing_state.lower().split(), lemm_lng_des[ind].split()) == 1,arr)),priorityOrder))

        for arr in filterWithState:
            if (len(arr) == 0):
                continue
            for item in arr:
                return item
        

    for arr in priorityOrder:
            if (len(arr) == 0):
                continue
            for item in arr:
                return item
    return -1
    
print(lng_des[findIng2('pea','cut')])

Peas, edible-podded, raw


In [196]:
print(lemm_lng_des[1911])
DistBatra('canned', lemm_lng_des[1911])

apple , raw , without skin , cooked , boiled


1.0

In [166]:
lng_des[1909]

'Apples, raw, with skin'

In [94]:
with open('ndbno-out.csv', 'w') as csvOutput:
    writer = csv.writer(csvOutput, delimiter=',', quotechar='"')
    with open('recipe-ing-mappings.csv', 'rt') as f:
        data = csv.reader(f)
        cnt = 0
        for row in data:
            cnt = cnt + 1

            row.append(ndb_nos[findIng(row[2])])
            row.append(lng_des[findIng(row[2])])
            writer.writerow(row)
#             if cnt == 20001:
            break
#         print(row[2], lng_des[findIng(row[2])])



['2610', ' 3 cups water', ' water', ' 3', ' cups', '', '', '', '']


In [13]:
DistBatra('a b', 'a')

0.3333333333333333

In [12]:
def DistBatra(str1, str2):
    str1 = set(str1.split())
    str2 = set(str2)
    return float(len(str1 & str2)) / len(str1)


In [200]:
def DistBatra(str1, str2):
    str1 = set(str1.split())
    str2 = set(str2.split())
    return float(len(str1 & str2)) / len(str1)
DistBatra('red lentil', 'lentil pink or red', )

{'lentil', 'red'}


1.0

In [7]:
def findIng(ing_name, ing_state=None):
    word_list = nltk.word_tokenize(ing_name)
    ing_name = ' '.join([wnl.lemmatize(w) for w in word_list]).strip()
    
    for i,d in enumerate(names_only):
        if DistBatra(ing_name.split(), d['0'].split()) == 1.0:
            return i
    for i,d in enumerate(names_only):
        if DistBatra(ing_name.split(), d['1'].split()) == 1.0:
            return i
        if DistBatra(ing_name.split(), d['0-1'].split()) == 1.0:
            return i
    for i,d in enumerate(names_only):
        if DistBatra(ing_name.split(), d['2'].split()) == 1.0:
            return i
        if DistBatra(ing_name.split(), d['0-2'].split()) == 1.0:
            return i
        if DistBatra(ing_name.split(), d['1-2'].split()) == 1.0:
            return i
    
    maxSoFar = 0
    maxi = -1
    for i,d in enumerate(names_only):
        if DistBatra(ing_name.split(), d['0'].split()) > maxSoFar:
            maxSoFar = DistBatra(ing_name.split(), d['0'].split())
            maxi = i
        if DistBatra(ing_name.split(), d['1'].split()) > maxSoFar:
            maxSoFar = DistBatra(ing_name.split(), d['1'].split())
            maxi = i
        if DistBatra(ing_name.split(), d['0-1'].split()) > maxSoFar:
            maxSoFar = DistBatra(ing_name.split(), d['0-1'].split())
            maxi = i
        if DistBatra(ing_name.split(), d['2'].split()) > maxSoFar:
            maxSoFar = DistBatra(ing_name.split(), d['2'].split())
            maxi = i
        if DistBatra(ing_name.split(), d['0-2'].split()) > maxSoFar:
            maxSoFar = DistBatra(ing_name.split(), d['0-2'].split())
            maxi = i
        if DistBatra(ing_name.split(), d['1-2'].split()) > maxSoFar:
            maxSoFar = DistBatra(ing_name.split(), d['1-2'].split())
            maxi = i
    return maxi

In [28]:
wnl.lemmatize('Apples')

'Apples'